In [386]:
from aiida.orm import load_node,load_code, load_group, Group
from aiida.plugins import WorkflowFactory, DataFactory
import aiida 
from aiida.engine import submit


aiida.load_profile()

Profile<uuid='960a408e945248238ac64a4982cfc6d7' name='dev'>

In [387]:
group_name = 'workchain/test/relax'
group = load_group(group_name)

successful_nodes = []

for node in group.nodes:
    workchain_node = load_node(node.pk)
    if node.is_finished:
        if node.is_finished_ok:
            successful_nodes.append(node.pk)

In [388]:
successful_nodes[:5]

[74644, 74684, 74707, 74733, 74756]

In [391]:
## Run the symmetry analysis with Quantum Espresso for the relaxed compounds.

if False: 
    code = load_code(92403)
    PwRelaxWorkChain = WorkflowFactory('quantumespresso.pw.relax')

    group_label = 'QE/test/symmetry'
    group, _ = Group.objects.get_or_create(label=group_label)
        
        
    for node in successful_nodes:
        
        workchain_node = load_node(node)
        
        structure = workchain_node.outputs.output_structure

        builder = PwRelaxWorkChain.get_builder_from_protocol(code=code, structure=structure)


        builder.base_init_relax.pw.metadata.options.withmpi = False
        builder.base_init_relax.pw.metadata.options.prepend_text = 'touch aiida.EXIT'

        builder.base_relax.pw.metadata.options.withmpi = False
        builder.base_relax.pw.metadata.options.prepend_text = 'touch aiida.EXIT'

        workchain_node = submit(builder)
        
        group.add_nodes(workchain_node)

        print(f"Workchain node {workchain_node.pk} added to group '{group_label}'")

In [392]:
def find_symmetries_matrix_number(filename):
    symmetries_matrix_number = []

    for j, i in enumerate(filename):
        if 'isym' in i:
            # Ensure there are at least two more elements ahead of the current index
            if j + 2 < len(filename):
                symmetries_matrix_number.append(filename[j + 2])
    
    return symmetries_matrix_number, len(symmetries_matrix_number)

In [393]:
## Here I will try to go over the the symmetry analysis that we did and extract the number of the symmetries 
## and their labels, so I can compare after the refinement of the cell.

group_label = 'QE/test/symmetry'
workchain_group = Group.objects.get(label=group_label)

# Iterate over all nodes in the group
for node in workchain_group.nodes:
    workchain_node = load_node(node.id)
    
    
    for descendant in workchain_node.called_descendants:
        if 'CalcJobNode.' in descendant.node_type:
            calcchain_node = descendant.id
            calculation = load_node(calcchain_node)
            retrieved = calculation.outputs.retrieved
            aiida_out = retrieved.base.repository.get_object_content('aiida.out').split()
            
            matrices_names , symmetry_count =  find_symmetries_matrix_number(aiida_out)
            
            print(node.id, matrices_names,symmetry_count)

100954 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'] 12
100969 ['1', '2'] 2
100987 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'] 16
101002 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'] 16
101020 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48'] 48
101037 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'] 12
101055 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'] 24
101071 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'] 16
101090 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '

In [403]:
p = load_node(100969)
structure = load_node(p.inputs.structure.id)

print('The positions are :')
print(structure.get_ase().get_positions())

print('The cell is :')
print(structure.get_ase().get_cell())

The positions are :
[[ 3.45419856e+00 -3.71960259e-19  1.61574061e+00]
 [ 4.28532431e+00 -1.38857254e-18  5.41404913e+00]
 [ 4.00998141e-02  1.10402313e-16 -4.83272612e-02]
 [ 1.61669672e+00 -7.79917821e-19  2.98420599e+00]]
The cell is :
Cell([[2.6440296907402, 1.5208433365876, 0.18969858181813], [-2.6440296907402, 1.5208433365876, -0.18969858181813], [-1.2461390856753, -2.0722828791801e-18, 7.5315918942124]])


In [405]:
from aiida.orm import load_node
import seekpath
import numpy as np

# Load the node and structure
p = load_node(100969)
structure = load_node(p.inputs.structure.id)

# Convert StructureData to ASE Atoms object
ase_structure = structure.get_ase()

# Extract the necessary information from the ASE Atoms object
lattice = ase_structure.cell.array

positions = ase_structure.get_scaled_positions()
numbers = ase_structure.get_atomic_numbers()

# Create the structure in the format required by seekpath
spglib_structure = (lattice, positions, numbers)

# Get the explicit k-path using seekpath
k_path = seekpath.get_explicit_k_path(structure=spglib_structure)
print('The spgroup is : ', k_path['spacegroup_number'])

The spgroup is :  8


In [414]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


pmg_structure = structure.get_pymatgen()
spg = SpacegroupAnalyzer(pmg_structure)
len(spg.get_symmetry_operations())

2

In [416]:
pmg_structure

Structure Summary
Lattice
    abc : 3.056115673841949 3.056115673841949 7.633985792612768
 angles : 85.41224988797036 94.58775011202964 120.31211717114172
 volume : 61.29042265493131
      A : 2.6440296907402 1.5208433365876 0.18969858181813
      B : -2.6440296907402 1.5208433365876 -0.18969858181813
      C : -1.2461390856753 -2.0722828791801e-18 7.5315918942124
    pbc : True True True
PeriodicSite: Cr (3.454, -3.72e-19, 1.616) [0.6955, -0.6955, 0.1795]
PeriodicSite: Ag (4.285, -1.389e-18, 5.414) [0.9683, -0.9683, 0.6701]
PeriodicSite: S (0.0401, 1.11e-16, -0.04833) [0.006, -0.006, -0.006719]
PeriodicSite: S (1.617, -7.799e-19, 2.984) [0.3944, -0.3944, 0.3764]

In [433]:
for i in spg.get_symmetry_operations(): 
    print(i.rotation_matrix)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]


In [396]:
k_path['conv_lattice']

array([[3.42995958, 0.        , 0.        ],
       [0.        , 3.42995958, 0.        ],
       [0.        , 0.        , 3.42995958]])

In [397]:
k_path['primitive_lattice']

array([[3.42995958, 0.        , 0.        ],
       [0.        , 3.42995958, 0.        ],
       [0.        , 0.        , 3.42995958]])

In [398]:
k_path['conv_positions']

array([[0. , 0. , 0. ],
       [0.5, 0.5, 0. ],
       [0.5, 0. , 0.5],
       [0. , 0.5, 0.5]])

In [399]:
k_path['primitive_positions']

array([[0. , 0. , 0. ],
       [0.5, 0.5, 0. ],
       [0.5, 0. , 0.5],
       [0. , 0.5, 0.5]])

In [420]:
import spglib
from aiida.orm import load_node
from pymatgen.core.periodic_table import Element
import numpy as np

# Load the crystal structure
p = load_node(100969)
structure = load_node(p.inputs.structure.id)

# Extract lattice vectors, atomic positions, and atomic symbols
lattice = np.array(structure.cell)  # Convert to numpy array
positions = [site.position for site in structure.sites]
symbols = [site.kind_name for site in structure.sites]

# Create the list of atomic numbers using PyMatGen
numbers = [Element(symbol).Z for symbol in symbols]

# Create the cell tuple
cell = (lattice, positions, numbers)

# Refine the cell
refined_cell_standardized_idealised = spglib.standardize_cell(cell,to_primitive=True,no_idealize=False)
refined_cell_standardized_no_idealised = spglib.standardize_cell(cell,to_primitive=True,no_idealize=True)

primitive_cell_standardized = spglib.find_primitive(cell)

# Extract refined lattice, positions, and numbers
refined_lattice_id, refined_positions_id, refined_numbers_id = refined_cell_standardized_idealised
refined_lattice_no_id, refined_positions_no_id, refined_numbers_no_id = refined_cell_standardized_no_idealised
primitive_lattice, primitive_positions, primitive_numbers = primitive_cell_standardized


print('################################################')
# Print the refined cell
print("Refined lattice vectors:")
print(refined_lattice_id)
print("Refined atomic positions:")
print(refined_positions_id)

print('################################################')
# Print the refined cell no idealised
print("Refined lattice vectors (no idealised) : ")
print(refined_lattice_no_id)
print("Refined atomic positions (no idealised): ")
print(refined_positions_no_id)


print('################################################')
# Print the refined cell no idealised
print("Primitive lattice vectors : ")
print(primitive_lattice)
print("Primitive atomic positions : ")
print(primitive_positions)

k_path = seekpath.get_explicit_k_path(structure=cell)
print(k_path['spacegroup_number'])

################################################
Refined lattice vectors:
[[ 3.04168667e+00  0.00000000e+00  0.00000000e+00]
 [-1.52084334e+00  2.65082601e+00  0.00000000e+00]
 [ 4.67446813e-16 -7.03967907e-01  7.60145830e+00]]
Refined atomic positions:
[[0.54580144 0.54580144 0.38425939]
 [0.71467569 0.71467569 0.58595087]
 [0.95990019 0.95990019 0.04832726]
 [0.38330328 0.38330328 0.01579401]]
################################################
Refined lattice vectors (no idealised) : 
[[ 0.00000000e+00 -3.04168667e+00  0.00000000e+00]
 [-2.64402969e+00  1.52084334e+00 -1.89698582e-01]
 [ 1.24613909e+00  2.07228288e-18 -7.53159189e+00]]
Refined atomic positions (no idealised): 
[[0.54580144 0.54580144 0.38425939]
 [0.71467569 0.71467569 0.58595087]
 [0.95990019 0.95990019 0.04832726]
 [0.38330328 0.38330328 0.01579401]]
################################################
Primitive lattice vectors : 
[[ 3.04168667e+00  0.00000000e+00  0.00000000e+00]
 [-1.52084334e+00  2.65082601e+00  0.000